# OpenAI Quickstart Guide

From the official documentation available at: https://platform.openai.com/docs/overview

You can use different models depending on your needs, check them at: https://platform.openai.com/docs/models

Note: check the pricing before using a model! --> https://platform.openai.com/docs/pricing

In [4]:
# Import required libraries
from dotenv import load_dotenv
import os
import openai
from pprint import pprint

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is set
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

# Set OpenAI API key
openai.api_key = api_key

# Initialize OpenAI client
from openai import OpenAI
client = OpenAI()

# Make a request to the model
response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

# Print the model's response
print("Response = ")
pprint(dict(response))
print(f"\nresponse.output_text = \n{response.output_text}")

Response = 
{'background': False,
 'conversation': None,
 'created_at': 1756758942.0,
 'error': None,
 'id': 'resp_68b6039e7fdc81a0b150d4d4922abe550f41a2da444eb905',
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'metadata': {},
 'model': 'gpt-4.1-2025-04-14',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_68b6039edeb481a0854054a272e196440f41a2da444eb905', content=[ResponseOutputText(annotations=[], text='Under the silvery moonlight, a gentle unicorn tiptoed through a field of stars, sprinkling sweet dreams over every sleeping child.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')],
 'parallel_tool_calls': True,
 'previous_response_id': None,
 'prompt': None,
 'prompt_cache_key': None,
 'reasoning': Reasoning(effort=None, generate_summary=None, summary=None),
 'safety_identifier': None,
 'service_tier': 'default',
 'status': 'completed',
 'store': True,
 'temperature'

You can specify instructions to provide high-level instructions adopted as an overall context for your prompts.

In [8]:
# Example of using the 'instructions' and 'reasoning' parameters in the model request
response_with_instructions = client.responses.create(
    model="gpt-4.1",
    input="Write a poem about a butterfly.",
    instructions="Answer in rhyme and with a cheerful style.")

# With some models you can also specify the reasoning effort parameter, e.g.    
# reasoning={"effort": "low"})
  
print("Response with instructions = ")
pprint(dict(response_with_instructions))
print(f"\nresponse_with_instructions.output_text = \n{response_with_instructions.output_text}")

Response with instructions = 
{'background': False,
 'conversation': None,
 'created_at': 1756760035.0,
 'error': None,
 'id': 'resp_68b607e34f208197ab5c49642b94e1840d7e0e684004ad84',
 'incomplete_details': None,
 'instructions': 'Answer in rhyme and with a cheerful style.',
 'max_output_tokens': None,
 'max_tool_calls': None,
 'metadata': {},
 'model': 'gpt-4.1-2025-04-14',
 'object': 'response',
 'output': [ResponseOutputMessage(id='msg_68b607e3e97c8197abf77607bb79c85d0d7e0e684004ad84', content=[ResponseOutputText(annotations=[], text='A butterfly flutters in sunlight so bright,  \nWings painted with splashes of pure delight.  \nIt dances on breezes, a soft, gentle flight,  \nA whisper of wonder, a beautiful sight.\n\nFrom blossom to blossom it skips on the way,  \nPainting the garden with colors of May.  \nIt sips from the nectar, then giggles with glee,  \nA speckled-winged artist as light as can be.\n\nWith silken wings open, it glimmers and spins,  \nA jewel on a breeze where the

## Structured Output

Structured Output allows you to receive responses from the model in a predefined format, such as JSON or other structured data types. This is useful when you need the model's output to be machine-readable for further processing, integration, or automation. By specifying the desired structure, you can ensure consistency and make it easier to extract specific information from the model's response.

In [10]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

response = client.responses.parse(
    model="gpt-5-mini",
    input=[
        {
            "role": "system",
            "content": "You are a helpful math tutor. Guide the user through the solution step by step.",
        },
        {"role": "user", "content": "how can I solve 8x + 7 = -23"},
    ],
    text_format=MathReasoning,
)

math_reasoning = dict(response.output_parsed)
print("math_reasoning = \n")
pprint(math_reasoning)


math_reasoning = 

{'final_answer': 'x = -15/4 (which equals -3.75)',
 'steps': [Step(explanation='Start with the equation and isolate the term containing x by subtracting 7 from both sides.', output='8x + 7 = -23  ⇒  8x = -23 - 7 = -30'),
           Step(explanation='Now solve for x by dividing both sides by 8.', output='x = -30/8'),
           Step(explanation='Simplify the fraction by dividing numerator and denominator by 2.', output='x = -15/4 = -3.75'),
           Step(explanation='Check the solution by substituting x back into the original equation: compute 8x + 7 with x = -15/4.', output='8(−15/4) + 7 = −30 + 7 = −23, which matches the original right-hand side.')]}
